In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___healthy', 'Corn_(maize)___Northern_Leaf_Blight', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
# Load the images and labels
filename = []
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        filename.append(img_file)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(11217, 128, 128, 3)

In [8]:
y.shape

(11217, 6)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, filename_train, filename_test = train_test_split(X, y, filename, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(8973, 128, 128, 3)

In [11]:
X_test.shape

(2244, 128, 128, 3)

In [12]:
y_train.shape

(8973, 6)

In [13]:
y_test.shape

(2244, 6)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

6

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-31 16:01:25.997712: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


281/281 [==============================] - 51s 178ms/step - loss: 0.4228 - accuracy: 0.8433 - val_loss: 0.5544 - val_accuracy: 0.7981
Epoch 2/10
281/281 [==============================] - 51s 181ms/step - loss: 0.1752 - accuracy: 0.9359 - val_loss: 0.3295 - val_accuracy: 0.8881
Epoch 3/10
281/281 [==============================] - 54s 191ms/step - loss: 0.1238 - accuracy: 0.9546 - val_loss: 0.1223 - val_accuracy: 0.9599
Epoch 4/10
281/281 [==============================] - 69s 246ms/step - loss: 0.1129 - accuracy: 0.9601 - val_loss: 0.3763 - val_accuracy: 0.8788
Epoch 5/10
281/281 [==============================] - 107s 381ms/step - loss: 0.0939 - accuracy: 0.9676 - val_loss: 0.1438 - val_accuracy: 0.9514
Epoch 6/10
281/281 [==============================] - 127s 452ms/step - loss: 0.0844 - accuracy: 0.9704 - val_loss: 0.1618 - val_accuracy: 0.9296
Epoch 7/10
281/281 [==============================] - 126s 449ms/step - loss: 0.0697 - accuracy: 0.9751 - val_loss: 0.4373 - val_accuracy: 

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 96.43%


In [19]:
cnn_scores

[0.11541353911161423, 0.9643493890762329]

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

71/71 [==============================] - 8s 106ms/step
Accuracy: 96.43%
Precision: 96.49%
Recall: 96.43%
F1 score: 96.43%


In [21]:
from tabulate import tabulate

# Create a DataFrame with true labels, predicted labels, and target labels
df = pd.DataFrame({'Filename': filename_test,
                   'True Labels': [target_labels[label] for label in y_test_classes],
                   'Predicted Labels': [target_labels[label] for label in y_pred]})

# Convert DataFrame to tabular format
table = tabulate(df, headers='keys', tablefmt='psql')

# Print the table
print(table)

# Save the DataFrame to a CSV file
df.to_csv('labels_predictions-cnn-jagung+pepper.csv', index=False)

+------+-------------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|      | Filename                                                                      | True Labels                                        | Predicted Labels                                   |
|------+-------------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------|
|    0 | 98327d43-c692-4922-82e3-a4e0db73ae40___JR_HL 8483.JPG                         | Pepper,_bell___healthy                             | Pepper,_bell___healthy                             |
|    1 | RS_Rust 2762.JPG                                                              | Corn_(maize)___Common_rust_                        | Corn_(maize)___Common_rust_                        |
|    2 | 348aafc3-4309-45

In [22]:
# Filter rows where true label and predicted label do not match
false_predictions = df[df['True Labels'] != df['Predicted Labels']]

# Convert DataFrame to tabular format
table = tabulate(false_predictions, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table)

# Save the false predictions to a CSV file
false_predictions.to_csv('false_predictions-cnn-jagung+pepper.csv', index=False)

False Predictions:
+------+------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|      | Filename                                                               | True Labels                                        | Predicted Labels                                   |
|------+------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------|
|   60 | f176a365-b5f0-4b3f-a4bc-d4d2c705b29e___JR_B.Spot 3263_flipTB.JPG       | Pepper,_bell___Bacterial_spot                      | Pepper,_bell___healthy                             |
|   78 | 2a42396f-f9a9-47cc-9dd7-7149faf1e0e2___JR_HL 8544_newPixel25.JPG       | Pepper,_bell___healthy                             | Corn_(maize)___healthy                             |
|   93 | a401fce7-28ad-4d2f-bf98-3ef87ae6

In [23]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

281/281 [==============================] - 32s 115ms/step


# rf

In [24]:
import time

In [25]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [26]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 379.7937002182007 seconds


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

71/71 [==============================] - 13s 175ms/step
Random Forest Classifier Accuracy: 97.91%
Random Forest Classifier Precision: 0.98
Random Forest Classifier Recall: 0.98
Random Forest Classifier F1 Score: 0.98


In [28]:
# Create a DataFrame with true labels and predicted labels
df_rf = pd.DataFrame({'Filename': filename_test,
                      'True Labels': [target_labels[label] for label in y_test_classes],
                      'Predicted Labels': [target_labels[label] for label in rf_predictions]})

# Convert DataFrame to tabular format
table_rf = tabulate(df_rf, headers='keys', tablefmt='psql')

# Print the table
print(table_rf)

# Save the DataFrame to a CSV file
df_rf.to_csv('labels_predictions-rf-jagung+pepper.csv', index=False)

+------+-------------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|      | Filename                                                                      | True Labels                                        | Predicted Labels                                   |
|------+-------------------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------|
|    0 | 98327d43-c692-4922-82e3-a4e0db73ae40___JR_HL 8483.JPG                         | Pepper,_bell___healthy                             | Pepper,_bell___healthy                             |
|    1 | RS_Rust 2762.JPG                                                              | Corn_(maize)___Common_rust_                        | Corn_(maize)___Common_rust_                        |
|    2 | 348aafc3-4309-45

In [29]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-jagung+pepper.csv', index=False)

False Predictions:
+------+--------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|      | Filename                                                           | True Labels                                        | Predicted Labels                                   |
|------+--------------------------------------------------------------------+----------------------------------------------------+----------------------------------------------------|
|   93 | a401fce7-28ad-4d2f-bf98-3ef87ae65ba5___RS_GLSp 4614.JPG            | Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot | Corn_(maize)___Northern_Leaf_Blight                |
|  202 | bbea95f8-cc26-40b6-9436-228f70d605d7___RS_GLSp 7332.JPG            | Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot | Corn_(maize)___Northern_Leaf_Blight                |
|  210 | f5eb9c66-7174-4f62-8261-7c7520601824___RS_GLSp 9342.